In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/"
N_FOLDS <- 10

In [4]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts0.03vs0_time_cross.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 91 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(91): SRR7817611 SRR7817612 ... SRR4317611 SRR4317612
colData names(3): Sample Dose sizeFactor


In [5]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [6]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()

In [7]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec0.03vs0_time_cross.Rds", sep = ""))
print(tcdd_dose_detail.vec)

 [1] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
[16] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
[31] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
[46] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.03 0.03 0.03 0.00 0.00 0.00
[61] 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.00 0.00 0.00 0.00 0.00 0.03 0.03
[76] 0.03 0.03 0.03 0.03 0.00 0.03 0.03 0.03 0.03 0.03 0.03 0.00 0.00 0.00 0.00
[91] 0.00


In [8]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls0.03vs0_time_cross.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [46] "ENSMUSG00000028777" "ENSMUSG00000028950" "ENSMUSG00000028971"
 [49] "ENSMUSG00000029064" "ENSMUSG00000029072" "ENSMUSG00000029236"
 [52] "ENSMUSG00000029371" "ENSMUSG00000029372" "ENSMUSG00000029373"
 [55] "ENSMUSG00000029379" "ENSMUSG00000029380" "ENSMUSG00000029417"
 [58] "ENSMUSG00000029530" "ENSMUSG00000029663" "ENSMUSG00000029713"
 [61] "ENSMUSG00000029819" "ENSMUSG00000031778" "ENSMUSG00000031861"
 [64] "ENSMUSG00000031980" "ENSMUSG00000032192" "ENSMUSG00000032562"
 [67] "ENSMUSG00000032766" "ENSMUSG00000033717" "ENSMUSG00000034117"
 [70] "ENSMUSG00000034855" "ENSMUSG00000035042" "ENSMUSG00000035383"
 [73] "ENSMUSG00000035431" "ENSMUSG00000035448" "ENSMUSG00000036353"
 [76] "ENSMUSG00000036362" "ENSMUSG00000036381" "ENSMUSG00000036402"
 [79] "ENSMUSG00000036437" "ENSMUSG00000036832" "ENSMUSG00000037010"
 [82] "ENSMUSG00000037014" "ENSMUSG00000037140" "ENSMUSG00000037346"
 [85] "ENSMUSG00000037944" "ENSMUSG00000038607" "ENSMUSG00000038668"
 [88] "ENSMUSG00000038811" "ENSMUS

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG00000037652" "ENSMUSG00000039989"
[13] "ENSMUSG00000040669"

$`R-MMU-4570463`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000031347"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-4570499`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020738" "ENSMUSG00000021546" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037652"
[13] "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MM

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

...

[1] "ENSMUSG00000033717" "ENSMUSG00000045318" "ENSMUSG00000058620"

$`R-MMU-400071`
[1] "ENSMUSG00000033717" "ENSMUSG00000045318"

$`R-MMU-9626914`
[1] "ENSMUSG00000033717" "ENSMUSG00000045318" "ENSMUSG00000058620"

$`R-MMU-1475414`
[1] "ENSMUSG00000033735"

$`R-MMU-1497853`
[1] "ENSMUSG00000033735"

$`R-MMU-1497869`
[1] "ENSMUSG00000033735"

$`R-MMU-1482545`
[1] "ENSMUSG00000033847"

$`R-MMU-1482571`
[1] "ENSMUSG00000033847"

$`R-MMU-1482629`
[1] "ENSMUSG00000033847"

$`R-MMU-1482696`
[1] "ENSMUSG00000033847"

$`R-MMU-1482816`
[1] "ENSMUSG00000033847" "ENSMUSG00000036257"

$`R-MMU-1482827`
[1] "ENSMUSG00000033847"

$`R-MMU-1482868`
[1] "ENSMUSG00000033847"

$`R-MMU-1482892`
[1] "ENSMUSG00000033847"

$`R-MMU-9603989`
[1] "ENSMUSG00000033849" "ENSMUSG00000034780" "ENSMUSG00000067370"
[4] "ENSMUSG00000074892"

$`R-MMU-1605530`
[1] "ENSMUSG00000033871" "ENSMUSG00000058440"

$`R-MMU-6813740`
[1] "ENSMUSG00000033917"

$`R-MMU-8953115`
[1] "ENSMUSG00000033943" "ENSMUSG00000038482" "ENSMUSG00

[1] "ENSMUSG00000036199" "ENSMUSG00000068329"

$`R-MMU-6813626`
[1] "ENSMUSG00000036216"

$`R-MMU-1482887`
[1] "ENSMUSG00000036257"

$`R-MMU-429101`
[1] "ENSMUSG00000036298"

$`R-MMU-2213239`
[1] "ENSMUSG00000036322" "ENSMUSG00000036594" "ENSMUSG00000060586"
[4] "ENSMUSG00000067341" "ENSMUSG00000073421"

$`R-MMU-417829`
[1] "ENSMUSG00000036353"

$`R-MMU-9611277`
[1] "ENSMUSG00000036353"

$`R-MMU-417843`
[1] "ENSMUSG00000036362"

$`R-MMU-417858`
[1] "ENSMUSG00000036381"

$`R-MMU-70471`
[1] "ENSMUSG00000036427"

$`R-MMU-70475`
[1] "ENSMUSG00000036427"

$`R-MMU-6781899`
[1] "ENSMUSG00000036432" "ENSMUSG00000056900"

$`R-NUL-9620900`
[1] "ENSMUSG00000036478"

$`R-MMU-1679098`
[1] "ENSMUSG00000036499" "ENSMUSG00000045322"

$`R-MMU-187895`
[1] "ENSMUSG00000036499" "ENSMUSG00000045322"

$`R-MMU-1168635`
[1] "ENSMUSG00000036526" "ENSMUSG00000052889"

$`R-MMU-9605682`
[1] "ENSMUSG00000036587"

$`R-MMU-5692480`
[1] "ENSMUSG00000036622"

$`R-MMU-446214`
[1] "ENSMUSG00000036632"

$`R-MMU-2730959`


[1] "ENSMUSG00000037341" "ENSMUSG00000060681"

$`R-MMU-426015`
[1] "ENSMUSG00000037341" "ENSMUSG00000039463"

$`R-MMU-390886`
[1] "ENSMUSG00000037346"

$`R-MMU-8869206`
[1] "ENSMUSG00000037366"

$`R-MMU-196955`
[1] "ENSMUSG00000037370"

$`R-MMU-391211`
[1] "ENSMUSG00000037424" "ENSMUSG00000056379" "ENSMUSG00000059763"
[4] "ENSMUSG00000100004" "ENSMUSG00000100186"

$`R-MMU-435366`
[1] "ENSMUSG00000037434"

$`R-MMU-8938300`
[1] "ENSMUSG00000037440"

$`R-MMU-196857`
[1] "ENSMUSG00000037514"

$`R-MMU-5696101`
[1] "ENSMUSG00000037542"

$`R-MMU-1474146`
[1] "ENSMUSG00000037580" "ENSMUSG00000046814"

$`R-MMU-1474158`
[1] "ENSMUSG00000037580" "ENSMUSG00000046814"

$`R-MMU-1296037`
[1] "ENSMUSG00000037610" "ENSMUSG00000054934" "ENSMUSG00000091091"

$`R-MMU-6797627`
[1] "ENSMUSG00000037686"

$`R-MMU-6786650`
[1] "ENSMUSG00000037697" "ENSMUSG00000061313"

$`R-MMU-449734`
[1] "ENSMUSG00000037722"

$`R-MMU-391940`
[1] "ENSMUSG00000037759" "ENSMUSG00000039942"

$`R-MMU-428625`
[1] "ENSMUSG0000003777

[1] "ENSMUSG00000039462"

$`R-MMU-5686304`
[1] "ENSMUSG00000039470" "ENSMUSG00000046841"

$`R-MMU-947499`
[1] "ENSMUSG00000039616"

$`R-MMU-893593`
[1] "ENSMUSG00000039648"

$`R-MMU-893596`
[1] "ENSMUSG00000039648"

$`R-MMU-893616`
[1] "ENSMUSG00000039648"

$`R-MMU-159431`
[1] "ENSMUSG00000039653"

$`R-MMU-192312`
[1] "ENSMUSG00000039653"

$`R-MMU-193491`
[1] "ENSMUSG00000039653"

$`R-MMU-6788650`
[1] "ENSMUSG00000039662"

$`R-MMU-373745`
[1] "ENSMUSG00000039683"

$`R-MMU-446208`
[1] "ENSMUSG00000039740"

$`R-MMU-448741`
[1] "ENSMUSG00000039760" "ENSMUSG00000074695"

$`R-MMU-71200`
[1] "ENSMUSG00000039783"

$`R-MMU-8854255`
[1] "ENSMUSG00000039813" "ENSMUSG00000079477"

$`R-MMU-442319`
[1] "ENSMUSG00000039878"

$`R-MMU-8875871`
[1] "ENSMUSG00000039908"

$`R-MMU-8879117`
[1] "ENSMUSG00000039952" "ENSMUSG00000066235"

$`R-MMU-9018768`
[1] "ENSMUSG00000039960"

$`R-MMU-9726862`
[1] "ENSMUSG00000039960"

$`R-MMU-391933`
[1] "ENSMUSG00000040016"

$`R-MMU-2995334`
[1] "ENSMUSG00000040018"

$

[1] "ENSMUSG00000042688"

$`R-MMU-3134954`
[1] "ENSMUSG00000042699"

$`R-MMU-197235`
[1] "ENSMUSG00000042751"

$`R-MMU-8939959`
[1] "ENSMUSG00000042751"

$`R-MMU-2161791`
[1] "ENSMUSG00000042808" "ENSMUSG00000063856" "ENSMUSG00000075706"

$`R-MMU-2161959`
[1] "ENSMUSG00000042808" "ENSMUSG00000063856" "ENSMUSG00000075706"

$`R-MMU-2161999`
[1] "ENSMUSG00000042808" "ENSMUSG00000063856" "ENSMUSG00000075706"

$`R-MMU-3341277`
[1] "ENSMUSG00000042808"

$`R-MMU-6805479`
[1] "ENSMUSG00000042854" "ENSMUSG00000059552"

$`R-MMU-9614414`
[1] "ENSMUSG00000042903" "ENSMUSG00000044167" "ENSMUSG00000048756"

$`R-MMU-9617657`
[1] "ENSMUSG00000042903"

$`R-MMU-9617711`
[1] "ENSMUSG00000042903"

$`R-MMU-9617755`
[1] "ENSMUSG00000042903"

$`R-NUL-9617710`
[1] "ENSMUSG00000042903"

$`R-NUL-9617742`
[1] "ENSMUSG00000042903"

$`R-NUL-9617757`
[1] "ENSMUSG00000042903"

$`R-NUL-9617903`
[1] "ENSMUSG00000042903"

$`R-NUL-9617912`
[1] "ENSMUSG00000042903"

$`R-MMU-187246`
[1] "ENSMUSG00000043013" "ENSMUSG000000

In [9]:
%%R
rxns <- rxn2ensembls.nls %>% names()

In [10]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [11]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

In [12]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

In [13]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

  10.738979  10.559562  10.743330  10.103340  10.104141
ENSMUSG00000011096  14.362000  14.262085  14.166008  14.388791  14.406380
ENSMUSG00000011148  13.442916  13.366018  13.541342  13.888804  13.881879
ENSMUSG00000011158  13.322957  13.292104  13.244827  13.252091  13.418479
ENSMUSG00000011171   9.151446   9.162291   9.162048   9.142753   9.579556
ENSMUSG00000011179  14.008465  13.891967  13.718644  13.803520  13.898170
ENSMUSG00000011256  10.773041  10.824745  10.607159  10.872302  10.442586
ENSMUSG00000011306  12.919352  12.891013  12.895732  13.116457  13.188978
ENSMUSG00000011463   9.151446   9.162291   9.162048   9.142753   9.142856
ENSMUSG00000011658  13.321534  12.894460  13.041604  13.153108  13.197873
ENSMUSG00000011751   9.151446   9.162291   9.162048   9.142753   9.142856
ENSMUSG00000011752  13.968507  13.783980  13.564719  13.504868  13.666814
ENSMUSG00000011837  12.478330  12.220543  12.302249  12.263054  12.530664
ENSMUSG00000011877  12.467387  12.352899  12.552625  12.

ENSMUSG00000003458  14.454852  14.587115  14.360717  14.660868  14.648515
ENSMUSG00000003464  15.989415  15.944953  15.865863  15.950210  15.840205
ENSMUSG00000003476   9.206870   9.732444   9.871533   9.166036   9.150890
ENSMUSG00000003477  19.380925  19.408986  19.728941  19.702457  19.441623
ENSMUSG00000003484  10.196817  10.216445   9.978379   9.870303   9.946178
ENSMUSG00000003500  10.752731  10.665280  10.689843  10.692172  11.030070
ENSMUSG00000003505   9.206870   9.206495   9.166275   9.166036   9.150890
ENSMUSG00000003518  14.026826  14.088845  14.293085  14.127829  14.146017
ENSMUSG00000003526  16.931227  16.990676  17.656302  17.612008  16.657409
ENSMUSG00000003528  16.912196  16.903658  16.958213  16.903415  17.032588
ENSMUSG00000003534  10.224729  10.885910   9.983309  10.408202  10.442125
ENSMUSG00000003541   9.206870  10.077442  10.357716  10.415977  10.684921
ENSMUSG00000003546  14.993250  15.068359  15.543336  15.433772  15.170665
ENSMUSG00000003549  10.770776  11.4054

ENSMUSG00000004789  16.753937  16.736615  16.715381  16.743917  16.794760
ENSMUSG00000004791   9.206870   9.206495   9.166275   9.166036   9.150890
ENSMUSG00000004815  14.399148  14.465342  14.205649  14.336243  14.262978
ENSMUSG00000004837  10.913981  11.233184  10.649632  10.787065  10.938892
ENSMUSG00000004843  14.125194  14.089035  13.760914  13.806136  14.205890
ENSMUSG00000004846  13.042445  12.777923  13.909727  13.680914  13.461434
ENSMUSG00000004849  13.402745  13.464838  13.753313  13.765037  13.765844
ENSMUSG00000004864   9.206870   9.845737  10.112343   9.678111  10.160722
ENSMUSG00000004865  13.147571  13.030494  13.239026  13.429346  13.563683
ENSMUSG00000004872   9.206870   9.206495   9.166275   9.166036   9.150890
ENSMUSG00000004880  12.487412  12.981051  12.073306  12.308996  12.539943
ENSMUSG00000004885   9.206870   9.206495   9.166275   9.166036   9.150890
ENSMUSG00000004892   9.206870   9.206495   9.166275   9.166036   9.150890
ENSMUSG00000004895  12.700863  12.7411

ENSMUSG00000005864   9.206870   9.206495   9.166275   9.166036   9.150890
ENSMUSG00000005871  14.257426  14.015980  13.857569  14.098062  14.217550
ENSMUSG00000005883   9.847238  10.077442   9.166275   9.166036  10.010384
ENSMUSG00000005886  14.802830  14.833429  14.947140  14.951757  15.093640
ENSMUSG00000005892   9.206870   9.206495   9.166275   9.166036   9.150890
ENSMUSG00000005893  13.934429  14.088464  14.000584  13.972249  14.329298
ENSMUSG00000005897  13.331342  12.992991  12.492800  12.931626  13.401739
ENSMUSG00000005899  13.061207  13.220725  13.007769  13.198309  12.965941
ENSMUSG00000005907  13.929358  13.890527  14.143530  13.917109  13.934385
ENSMUSG00000005947   9.928247   9.560028   9.871533   9.682905  10.093259
ENSMUSG00000005949  13.154034  13.425417  12.939554  13.232006  13.444749
ENSMUSG00000005950   9.206870   9.206495   9.871533   9.870303   9.150890
ENSMUSG00000005951  14.823557  14.946724  15.372590  15.298549  15.008229
ENSMUSG00000005952   9.206870   9.2064

ENSMUSG00000007613  13.879494  13.869899  13.600422  13.655023  13.664103
ENSMUSG00000007617  12.000145  12.577617  12.399455  12.065234  11.745052
ENSMUSG00000007646   9.206870   9.206495   9.871533   9.166036   9.616640
ENSMUSG00000007653   9.206870   9.206495   9.166275   9.166036   9.150890
ENSMUSG00000007655  11.121420  11.287586  10.423168  10.541732  10.580069
ENSMUSG00000007656  13.915330  13.866180  13.659758  13.709442  13.791250
ENSMUSG00000007659  14.023838  14.166297  13.864212  13.931333  13.645475
ENSMUSG00000007670  14.013034  14.047760  14.045963  14.187286  14.316936
ENSMUSG00000007682   9.206870   9.206495   9.166275   9.166036   9.150890
ENSMUSG00000007739  15.345724  15.289296  15.702085  15.577955  15.662221
ENSMUSG00000007812  14.221454  14.383133  14.494034  14.356737  14.828732
ENSMUSG00000007815  15.187205  15.132608  14.893264  15.048474  15.145797
ENSMUSG00000007827  11.527630  11.379609  11.668040  11.464560  11.832679
ENSMUSG00000007850  15.785320  15.7789

...

ENSMUSG00000017776  15.424309  15.101957  15.366063  15.349522  15.508912
ENSMUSG00000017778  16.254849  16.246882  16.312005  16.361723  16.130064
ENSMUSG00000017781  14.262223  14.130991  14.113065  14.082691  14.148141
ENSMUSG00000017831  14.614581  14.091992  14.623935  14.569515  14.466641
ENSMUSG00000017837  12.789847  13.022386  12.710619  12.973299  12.845933
ENSMUSG00000017843  14.493669  14.473757  14.478318  14.423494  14.614461
ENSMUSG00000017858  13.945150  13.822418  13.681369  13.762535  13.743282
ENSMUSG00000017861  10.069741   9.635795   9.714355   9.951463  10.240789
ENSMUSG00000017868  15.290724  14.572135  14.790391  14.956110  14.849882
ENSMUSG00000017897   9.140427   9.154494   9.155088   9.641309   9.755839
ENSMUSG00000017929  13.574630  13.491701  13.285504  13.067125  13.214309
ENSMUSG00000017950  18.312591  17.877591  17.757338  17.779504  18.267992
ENSMUSG00000017969  10.461531  11.196513  10.525087  10.334794  10.646648
ENSMUSG00000018001  11.909434  11.6077

ENSMUSG00000001986  10.518210
ENSMUSG00000001998  12.114419
ENSMUSG00000001999  12.653023
ENSMUSG00000002010  14.410174
ENSMUSG00000002015  16.118304
ENSMUSG00000002020  10.757788
ENSMUSG00000002028  13.928177
ENSMUSG00000002031  13.636467
ENSMUSG00000002033  10.389875
ENSMUSG00000002043  12.447823
ENSMUSG00000002052  14.976639
ENSMUSG00000002059  11.019637
ENSMUSG00000002068  10.225975
ENSMUSG00000002100   9.145019
ENSMUSG00000002102  15.741425
ENSMUSG00000002108  15.606002
ENSMUSG00000002109  12.904579
ENSMUSG00000002111  11.879304
ENSMUSG00000002129  13.635804
ENSMUSG00000002147  14.819120
ENSMUSG00000002204  10.288826
ENSMUSG00000002205  13.724369
ENSMUSG00000002210  12.162958
ENSMUSG00000002221  13.228024
ENSMUSG00000002227  13.233107
ENSMUSG00000002233  11.926541
ENSMUSG00000002249  12.188618
ENSMUSG00000002250  12.414324
ENSMUSG00000002257   9.873555
ENSMUSG00000002279  14.208435
ENSMUSG00000002289  16.494796
ENSMUSG00000002297  10.974791
ENSMUSG00000002307  12.789327
ENSMUSG000

ENSMUSG00000004933  10.385302
ENSMUSG00000004934  12.321494
ENSMUSG00000004936  14.176894
ENSMUSG00000004937  14.760789
ENSMUSG00000004939   9.145019
ENSMUSG00000004947  11.544213
ENSMUSG00000004948   9.145019
ENSMUSG00000004951  11.030356
ENSMUSG00000004952  11.083554
ENSMUSG00000004961   9.145019
ENSMUSG00000004980  17.435938
ENSMUSG00000004988   9.596821
ENSMUSG00000004996  12.397518
ENSMUSG00000005034  14.784088
ENSMUSG00000005043  12.009294
ENSMUSG00000005054  13.693857
ENSMUSG00000005057  10.391697
ENSMUSG00000005069  16.052868
ENSMUSG00000005087  12.978471
ENSMUSG00000005089  16.934395
ENSMUSG00000005102  12.208774
ENSMUSG00000005103  14.326235
ENSMUSG00000005107  14.514193
ENSMUSG00000005142  16.175745
ENSMUSG00000005148  10.050838
ENSMUSG00000005150  13.544865
ENSMUSG00000005161  15.608878
ENSMUSG00000005198  14.642907
ENSMUSG00000005204  14.130513
ENSMUSG00000005220   9.761075
ENSMUSG00000005225  13.880860
ENSMUSG00000005232   9.366326
ENSMUSG00000005233  11.751497
ENSMUSG000

ENSMUSG00000009394   9.761075
ENSMUSG00000009406  11.831200
ENSMUSG00000009470  15.396503
ENSMUSG00000009487   9.145019
ENSMUSG00000009535  13.188908
ENSMUSG00000009545  10.464826
ENSMUSG00000009549  13.863016
ENSMUSG00000009555  13.712486
ENSMUSG00000009566  17.261214
ENSMUSG00000009575  14.362052
ENSMUSG00000009585  11.490010
ENSMUSG00000009588   9.145019
ENSMUSG00000009614  18.708696
ENSMUSG00000009621  14.455422
ENSMUSG00000009630  13.181323
ENSMUSG00000009647  13.755990
ENSMUSG00000009681  13.909658
ENSMUSG00000009731   9.761075
ENSMUSG00000009863  16.587735
ENSMUSG00000009900   9.145019
ENSMUSG00000009905  14.300515
ENSMUSG00000009907  14.471514
ENSMUSG00000009927  15.658262
ENSMUSG00000009941   9.145019
ENSMUSG00000009995  14.234088
ENSMUSG00000010021   9.873555
ENSMUSG00000010025  17.572083
ENSMUSG00000010045  13.515878
ENSMUSG00000010047  14.557341
ENSMUSG00000010051  15.134232
ENSMUSG00000010057  12.394933
ENSMUSG00000010064  18.679156
ENSMUSG00000010066   9.145019
ENSMUSG000

ENSMUSG00000017057  14.311077
ENSMUSG00000017119  16.752905
ENSMUSG00000017132  14.454238
ENSMUSG00000017144  12.828986
ENSMUSG00000017146  10.901537
ENSMUSG00000017165   9.145019
ENSMUSG00000017167  11.154622
ENSMUSG00000017176  10.977326
ENSMUSG00000017210  13.201638
ENSMUSG00000017221  15.250028
ENSMUSG00000017264  13.360539
ENSMUSG00000017288  13.857244
ENSMUSG00000017291  14.822658
ENSMUSG00000017300   9.566219
ENSMUSG00000017307  13.256147
ENSMUSG00000017309  13.253761
ENSMUSG00000017311   9.145019
ENSMUSG00000017316   9.145019
ENSMUSG00000017344  20.410829
ENSMUSG00000017376  12.940061
ENSMUSG00000017390  13.093566
ENSMUSG00000017404  14.568685
ENSMUSG00000017412  11.495239
ENSMUSG00000017428  15.830814
ENSMUSG00000017453  17.039070
ENSMUSG00000017466  12.104489
ENSMUSG00000017485  14.414232
ENSMUSG00000017491  11.048178
ENSMUSG00000017493  18.930345
ENSMUSG00000017499  10.120889
ENSMUSG00000017548  13.218666
ENSMUSG00000017588   9.145019
ENSMUSG00000017607  10.515852
ENSMUSG000

In [14]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

      Resample1
 [1,]         1
 [2,]         2
 [3,]         3
 [4,]         4
 [5,]         5
 [6,]         6
 [7,]         7
 [8,]         8
 [9,]         9
[10,]        10
[11,]        11
[12,]        12
[13,]        13

In [15]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)


ENSMUSG00000009585  11.385005  10.544146  10.483705  11.348633  11.116632
ENSMUSG00000009588   9.150859   9.157001   9.157071   9.154204   9.154303
ENSMUSG00000009614  18.519163  18.338602  18.297744  18.588483  18.555424
ENSMUSG00000009621  14.797548  14.818562  14.754075  14.560110  14.646305
ENSMUSG00000009630  13.906451  13.548661  13.539693  13.439865  13.558351
ENSMUSG00000009647  12.545312  12.812478  12.884587  12.461561  12.723972
ENSMUSG00000009681  13.935715  13.805484  13.793830  13.860959  13.978107
ENSMUSG00000009731  10.093032   9.945168  10.049568   9.154204   9.626029
ENSMUSG00000009863  17.016363  16.831142  16.801126  16.836201  16.861659
ENSMUSG00000009900   9.150859   9.157001   9.157071   9.154204   9.154303
ENSMUSG00000009905  13.541343  13.538147  13.515907  14.001994  13.860588
ENSMUSG00000009907  14.747482  14.857550  14.889399  14.605553  14.685305
ENSMUSG00000009927  15.047041  14.959273  15.022949  14.867479  14.858230
ENSMUSG00000009941   9.150859   9.157

ENSMUSG00000002602  12.456185  12.412574  11.992434  14.132554  14.109664
ENSMUSG00000002603  11.380781  11.183970  11.598224  12.699388  12.570454
ENSMUSG00000002625  15.001595  15.023573  14.456427  14.199166  14.258123
ENSMUSG00000002633   9.583059   9.141440   9.943807  10.123312  10.167959
ENSMUSG00000002658  13.334998  13.026249  13.715582  13.842748  13.831964
ENSMUSG00000002660  14.658823  14.721227  14.752935  14.627800  14.596195
ENSMUSG00000002664   9.141691   9.141440   9.156026  10.097568   9.698665
ENSMUSG00000002668  11.206445  10.971024  10.725093  11.679564  11.542406
ENSMUSG00000002679  12.284080  11.988976  12.040055  12.083899  12.024189
ENSMUSG00000002699  10.894039  10.865906  10.434332  12.325497  12.433118
ENSMUSG00000002731  13.257193  13.190918  12.971258  12.052434  12.325190
ENSMUSG00000002733  13.285729  13.127482  13.246377  13.041046  13.352851
ENSMUSG00000002741  14.581338  14.715931  14.884198  14.663021  14.718152
ENSMUSG00000002748  15.528756  15.5681

ENSMUSG00000003623  17.191625  17.179494  17.150576  17.182672  17.184835
ENSMUSG00000003644  12.695457  12.529013  12.977144  13.032161  13.074116
ENSMUSG00000003660  15.353001  15.377200  15.123680  15.153222  15.121505
ENSMUSG00000003665   9.141691   9.141440   9.156026   9.598114   9.143358
ENSMUSG00000003731  14.848970  14.737791  14.835766  14.797303  14.610551
ENSMUSG00000003746  17.086049  17.060846  16.845750  17.374874  17.380847
ENSMUSG00000003752  12.097194  12.255653  12.231876  12.213020  12.238599
ENSMUSG00000003779  11.347338  11.035262  10.686241  11.143047  11.286694
ENSMUSG00000003809  18.399694  18.405418  18.437012  17.764596  17.750522
ENSMUSG00000003812  13.205418  13.409166  13.420060  13.439915  13.412509
ENSMUSG00000003813  14.870437  14.952590  14.824057  14.668106  14.669631
ENSMUSG00000003814  17.910916  17.920847  17.910619  18.153154  18.203184
ENSMUSG00000003824  11.906676  11.915219  11.738351  11.374753  11.567034
ENSMUSG00000003848  12.625770  12.6438

ENSMUSG00000004937  15.194873  15.213498  15.025363  15.071122  15.041876
ENSMUSG00000004939   9.141691   9.141440   9.156026   9.145332   9.143358
ENSMUSG00000004947  11.695433  11.773344  11.658176  11.496042  11.292112
ENSMUSG00000004948   9.141691   9.141440   9.156026   9.145332   9.143358
ENSMUSG00000004951  11.293834  11.057139  11.110217  12.078818  12.121079
ENSMUSG00000004952  11.026340  10.841777  10.258260  12.122577  12.134433
ENSMUSG00000004961   9.141691   9.141440   9.642071   9.145332   9.143358
ENSMUSG00000004980  17.493486  17.503203  17.451051  17.334695  17.270694
ENSMUSG00000004988   9.357822   9.627549   9.156026   9.145332   9.143358
ENSMUSG00000004996  13.355630  13.571511  13.599503  13.114402  13.111386
ENSMUSG00000005034  14.929529  15.124736  14.921032  14.609978  14.547934
ENSMUSG00000005043  11.382891  11.572540  12.045798  12.464611  12.318951
ENSMUSG00000005054  13.862950  13.721910  13.880170  13.535425  13.775081
ENSMUSG00000005057  10.979892  11.4222

ENSMUSG00000006024  15.463033  15.504033  15.532547  15.365039  15.439845
ENSMUSG00000006057  14.748902  14.628065  14.707065  14.650009  14.735932
ENSMUSG00000006058  13.549214  13.394208  13.616179  13.362116  13.449067
ENSMUSG00000006127  13.114385  13.394208  13.258108  12.962054  12.953069
ENSMUSG00000006134  14.134079  14.324384  14.104684  14.472882  14.327860
ENSMUSG00000006169  14.434336  14.505100  14.307451  14.760324  14.940269
ENSMUSG00000006205   9.949004  10.207534  10.663354  10.597049  10.280052
ENSMUSG00000006216   9.141691   9.141440   9.156026   9.145332   9.143358
ENSMUSG00000006219   9.947329  10.090579  10.205890  11.173556  11.168630
ENSMUSG00000006235  10.972771  10.610596  10.669125  10.676396  10.531697
ENSMUSG00000006262  15.085035  15.075239  14.859069  14.558187  14.675447
ENSMUSG00000006269   9.141691   9.141440   9.156026   9.145332   9.143358
ENSMUSG00000006273  14.260357  14.220879  13.915858  14.338422  14.224960
ENSMUSG00000006276  12.951724  13.1663

...

ENSMUSG00000017491  11.747486  12.352573  12.519024  11.000374  11.290905
ENSMUSG00000017493  18.872369  19.393071  19.390517  19.116357  19.518682
ENSMUSG00000017499  10.446681  10.443550  10.325434   9.624494   9.619631
ENSMUSG00000017548  14.421997  14.419984  14.405298  13.060375  13.393813
ENSMUSG00000017588   9.168863   9.168650   9.168764   9.104129   9.150556
ENSMUSG00000017607  10.374582  10.212287   9.884365  10.149813  10.229306
ENSMUSG00000017615  13.643766  13.818320  13.771163  14.190008  14.244036
ENSMUSG00000017631  11.390357  10.953518  11.309199  11.730460  11.696113
ENSMUSG00000017652  10.304738  10.019088  10.014957  10.134575   9.911015
ENSMUSG00000017670  12.149765  11.519730  11.909446  12.617537  12.287956
ENSMUSG00000017677  14.592514  14.276280  14.359795  14.360369  14.118356
ENSMUSG00000017686  14.586514  14.586529  14.657559  13.802532  14.005245
ENSMUSG00000017697  10.222563  10.128038  10.283494  10.700798  10.689148
ENSMUSG00000017707  16.595371  17.0268

ENSMUSG00000000561  13.877531  13.608396  13.610413  13.841354  13.859345
ENSMUSG00000000562   9.150509   9.136037   9.560059   9.626804   9.152302
ENSMUSG00000000563  16.309130  16.515409  16.505638  16.569617  16.607140
ENSMUSG00000000567  11.569823  12.006406  12.256889  11.325923  11.584070
ENSMUSG00000000568  15.100019  14.897444  14.832406  14.658235  14.836490
ENSMUSG00000000579  10.252316  10.049955  10.165057  10.103438  10.302029
ENSMUSG00000000581  14.755774  14.684886  14.648068  14.692356  14.753688
ENSMUSG00000000594  16.445679  16.302744  16.219316  16.442701  16.442993
ENSMUSG00000000605  14.894106  14.902309  14.860697  14.790080  14.767689
ENSMUSG00000000617   9.150509   9.136037   9.136151   9.152302   9.152302
ENSMUSG00000000627   9.816666   9.136037   9.560059   9.152302   9.152302
ENSMUSG00000000628  10.794559  11.190820  11.154961  10.881406  11.084384
ENSMUSG00000000673  18.025870  18.151346  18.127450  18.097757  18.114325
ENSMUSG00000000682  11.274996  11.7128

ENSMUSG00000001642  10.919744  10.884709  11.211630  11.028218  10.960010
ENSMUSG00000001656   9.150509   9.136037   9.136151   9.152302   9.152302
ENSMUSG00000001657   9.150509   9.136037   9.136151   9.152302   9.152302
ENSMUSG00000001663  17.114112  17.434380  17.421034  17.191631  17.188895
ENSMUSG00000001670  18.585638  18.661838  18.649468  18.654554  18.689130
ENSMUSG00000001729  14.536044  14.473755  14.350015  14.411309  14.574002
ENSMUSG00000001741  11.423996  11.246957  11.043049  10.985098  10.815880
ENSMUSG00000001750  14.743260  14.702970  14.732888  14.436530  14.435416
ENSMUSG00000001751  13.807298  13.310673  13.373823  13.682859  13.628431
ENSMUSG00000001755  15.844882  15.902500  15.902265  16.047187  16.020990
ENSMUSG00000001761  12.010736  11.970065  12.020589  11.964786  11.743675
ENSMUSG00000001767  12.724489  12.369121  12.436927  12.438866  12.478651
ENSMUSG00000001768  13.079344  12.941371  12.945314  12.530622  12.646565
ENSMUSG00000001773   9.150509   9.1360

ENSMUSG00000002803  12.700722  12.617345  12.712730  12.625998  12.328292
ENSMUSG00000002804  12.239570  12.248988  12.041752  12.040647  11.929124
ENSMUSG00000002814  11.970094  12.063810  12.114925  11.711709  11.974833
ENSMUSG00000002820  14.060944  14.056498  13.978653  14.101030  14.006465
ENSMUSG00000002846  13.890964  13.687099  13.624756  13.842407  13.709562
ENSMUSG00000002847  14.697954  14.711137  14.830495  14.571787  14.595210
ENSMUSG00000002870  11.132550  11.108108  11.251017  11.122754  11.401766
ENSMUSG00000002885  12.831057  13.007386  13.016056  13.128945  13.038148
ENSMUSG00000002900  12.194582  12.486694  12.224186  12.157732  11.713097
ENSMUSG00000002908  11.722485  11.392668  11.734607  11.656226  11.761607
ENSMUSG00000002944  14.714809  15.050482  15.108668  15.639561  15.598025
ENSMUSG00000002948  14.921508  14.856791  14.900990  14.814027  14.795200
ENSMUSG00000002957  15.555160  15.558859  15.568384  15.404742  15.353088
ENSMUSG00000002963  13.758520  13.7154

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

In [17]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train0.03vs0_time_cross.Rds",sep=""))

In [18]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

In [19]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

In [20]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [21]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"0.03vs0_time_cross.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

In [22]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls0.03vs0_time_cross.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls0.03vs0_time_cross.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6950302 371.2   13075682 698.4 11898195 635.5
Vcells 15944447 121.7   25842636 197.2 25842636 197.2


In [23]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls0.03vs0_time_cross.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train0.03vs0_time_cross.Rds", sep = ""))

In [24]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
}

In [25]:
%%R
# print(tissue_annotation)
# print(binary_tcdd_dose_annotations)
# print(paste("Processing tissue annotation:", tissue_annotation))
# print(length(binary_tcdd_dose_detail_vec.test.cv_test_list))
# print(sum(tcdd_dose_detail.vec.train.cv_test == tissue_annotation))
# print(length(rxns))

# binary_tcdd_dose_annotations <- c(0.00, 30.00, 0.01, 0.03, 0.10, 0.30, 1.00, 3.00, 10.00)

formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00" "0.03"
[1] "0.00"
[1] "0.03"


In [26]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
#        print(length(vst.count.mtx.train.cv_train))
#        print(length(gtex_tissue_detail.vec.train.cv_train))
       
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
#          print(tcdd_dose_detail.vec.train.cv_test)
#          print("-------------")
#          print(tissue_annotation)
#          print(binary_tcdd_dose_detail_vec.test.cv_test_list)
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
#        print(dim(cv_train.expr_mat))
#        print(length(gtex_tissue_detail.vec.train.cv_train))
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         # calculate misclassification rate (https://stat.ethz.ch/pipermail/r-help/2011-September/288885.html)
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  0.765545338814863 : Last ECOUNT =  178 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  0.660663366327256 : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  0.458874857496494 : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  0.361822902911183 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  0.199508643512385 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-194544 : Last ARI =  0.122800253670511 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last

[1] "Last RXN_ID =  R-MMU-8949580 : Last ARI =  0.69557813787421 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9692137 : Last ARI =  0.115666203578715 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-450580 : Last ARI =  0.9 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-6791222 : Last ARI =  0.9 : Last ECOUNT =  61 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9696268 : Last ARI =  0.929411764705882 : Last ECOUNT =  41 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1912394 : Last ARI =  0.283669943614201 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8936481 : 

[1] "Last RXN_ID =  R-MMU-3928599 : Last ARI =  0.158089306375484 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928625 : Last ARI =  0.4689792652936 : Last ECOUNT =  17 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928657 : Last ARI =  0.483339901612368 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9034539 : Last ARI =  0.0967437663102498 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-983156 : Last ARI =  0.955555555555555 : Last ECOUNT =  242 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-174119 : Last ARI =  0.581117970760249 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.4 % remaining..."
[1] "Las

[1] "Last RXN_ID =  R-MMU-3000411 : Last ARI =  0.860176991150442 : Last ECOUNT =  33 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-5228523 : Last ARI =  0.806330837304289 : Last ECOUNT =  33 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-2395328 : Last ARI =  0.132080713777237 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-8852134 : Last ARI =  0.9 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-354124 : Last ARI =  0.642405384701457 : Last ECOUNT =  13 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-429415 : Last ARI =  0.72013840473055 : Last ECOUNT =  13 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.4 % remaining..."
[1] "Last RXN_ID =  R-M

[1] "Last RXN_ID =  R-MMU-5682377 : Last ARI =  0.761616161616162 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-2426496 : Last ARI =  0.452468639266535 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.8 % remaining..."
[1] "Last RXN_ID =  R-NUL-2467092 : Last ARI =  0.306544458616845 : Last ECOUNT =  7 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-168162 : Last ARI =  0.379241434582536 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-445079 : Last ARI =  0.407061328491799 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-8856808 : Last ARI =  0.809719552015624 : Last ECOUNT =  19 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.4 % remaining..."
[1] "Last 

...

[1] "Last RXN_ID =  R-NUL-1504194 : Last ARI =  0.0679352695843237 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  20.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-437092 : Last ARI =  -0.0943742292704327 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  20 % remaining..."
[1] "Last RXN_ID =  R-MMU-173597 : Last ARI =  0.0336301010243523 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-5663185 : Last ARI =  0.423788798468888 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-111285 : Last ARI =  0.253842311950988 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9758661 : Last ARI =  1 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-4

[1] "Last RXN_ID =  R-MMU-5649725 : Last ARI =  0.133441286382463 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  15.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-9724745 : Last ARI =  0.167009893684819 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  15 % remaining..."
[1] "Last RXN_ID =  R-MMU-9010872 : Last ARI =  -0.00207520531656957 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  14.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-194641 : Last ARI =  0.322609999752306 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  14.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-202939 : Last ARI =  0.0449311269630563 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  14.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9831009 : Last ARI =  0.348364091884952 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  14.6 % remaining..."
[1] "Last 

[1] "Last RXN_ID =  R-MMU-109759 : Last ARI =  -0.0610305381030275 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  10.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-110352 : Last ARI =  0.0455203508527702 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  10 % remaining..."
[1] "Last RXN_ID =  R-MMU-975916 : Last ARI =  0.2063454081338 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-417858 : Last ARI =  0.0284507977606144 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-446214 : Last ARI =  -0.0151281548362831 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5578712 : Last ARI =  0.219705989941071 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.6 % remaining..."
[1] "Last RXN_ID 

[1] "Last RXN_ID =  R-MMU-9679045 : Last ARI =  0.180072283881278 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  5.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-965067 : Last ARI =  0.156412164212713 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  5 % remaining..."
[1] "Last RXN_ID =  R-MMU-451056 : Last ARI =  0.278589442217934 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  4.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-1640164 : Last ARI =  0.44195119773825 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  4.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-1482897 : Last ARI =  -0.0495170125049948 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  4.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5340195 : Last ARI =  0.0810853972911977 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  4.6 % remaining..."
[1] "Last RXN_ID =

[1] "Last RXN_ID =  R-MMU-5695989 : Last ARI =  -0.0381911447883509 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  0.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-6786789 : Last ARI =  0.22989939772989 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  0.1 % remaining..."


In [27]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls0.03vs0_time_cross.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls0.03vs0_time_cross.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls0.03vs0_time_cross.Rds", sep = ""))

In [28]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df0.03vs0_time_cross.Rds", sep = ""))

                          RXN_ID   MISCLASS           ARI ECOUNT
R-MMU-170666.0.00   R-MMU-170666 0.00000000  3.618223e-01     14
R-MMU-170666.0.03   R-MMU-170671 0.17500000  3.618223e-01     14
R-MMU-170671.0.00   R-MMU-170674 0.00000000  3.618223e-01     14
R-MMU-170671.0.03   R-MMU-170686 0.17500000  3.618223e-01     14
R-MMU-170674.0.00   R-MMU-380073 0.00000000  7.211009e-01    182
R-MMU-170674.0.03   R-MMU-392187 0.17500000  5.561540e-01     22
R-MMU-170686.0.00   R-MMU-392195 0.00000000  5.561540e-01     22
R-MMU-170686.0.03   R-MMU-392202 0.17500000  5.561540e-01     22
R-MMU-380073.0.00   R-MMU-749454 0.00000000  7.655453e-01    178
R-MMU-380073.0.03   R-MMU-749456 0.06583333  7.655453e-01    178
R-MMU-392187.0.00   R-MMU-751001 0.00000000  4.803846e-01     21
R-MMU-392187.0.03  R-MMU-8964229 0.11055556  2.344629e-01      5
R-MMU-392195.0.00  R-MMU-8964242 0.00000000  2.344629e-01      5
R-MMU-392195.0.03  R-MMU-8964329 0.11055556  2.344629e-01      5
R-MMU-392202.0.00  R-MMU-

R-MMU-205060.0.03  R-MMU-5654169 0.34444444  5.685248e-01     14
R-MMU-205099.0.00  R-MMU-5654222 0.00000000  2.975735e-01     12
R-MMU-205099.0.03  R-MMU-5654224 0.22722222  2.975735e-01     12
R-MMU-205112.0.00  R-MMU-5654392 0.00000000  1.937930e-01     20
R-MMU-205112.0.03  R-MMU-5654397 0.17277778  1.518254e-01     16
R-MMU-205117.0.00  R-MMU-5654399 0.00000000  1.518254e-01     16
R-MMU-205117.0.03  R-MMU-5654402 0.27722222  3.103498e-01     20
R-MMU-209532.0.00  R-MMU-5654404 0.00000000  1.088436e-01     16
R-MMU-209532.0.03  R-MMU-5654406 0.44083333  1.468448e-01     18
R-MMU-209566.0.00  R-MMU-5654407 0.00000000  1.088436e-01     16
R-MMU-209566.0.03  R-MMU-5654408 0.06583333  3.420179e-01     12
R-MMU-9012761.0.00 R-MMU-5654409 0.00000000  3.420179e-01     12
R-MMU-9012761.0.03 R-MMU-5654413 0.22916667  4.167083e-01     17
R-MMU-3247837.0.00 R-MMU-5654416 0.00000000  2.829295e-01     15
R-MMU-3247837.0.03 R-MMU-5654418 0.18250000  3.087711e-01     14
R-MMU-3247840.0.00 R-MMU-

R-MMU-5654169.0.03 R-MMU-9619682 0.10694444  1.362219e-01      6
R-MMU-5654222.0.00 R-MMU-9010527 0.00000000 -5.754891e-02      3
R-MMU-5654222.0.03 R-MMU-9011075 0.18861111  2.219157e-02      2
R-MMU-5654224.0.00 R-MMU-9011080 0.00000000  9.327847e-02      2
R-MMU-5654224.0.03 R-MMU-9011082 0.18861111  9.327847e-02      2
R-MMU-5654392.0.00 R-MMU-9011144 0.00000000  1.052195e-01      3
R-MMU-5654392.0.03 R-MMU-9011146 0.23638889  1.052195e-01      3
R-MMU-5654397.0.00 R-MMU-9753028 0.00000000  1.984607e-01      5
R-MMU-5654397.0.03 R-MMU-9753126 0.27583333  1.984607e-01      5
R-MMU-5654399.0.00  R-MMU-514604 0.00000000  9.611055e-02      2
R-MMU-5654399.0.03  R-MMU-975389 0.27583333  2.853411e-01      4
R-MMU-5654402.0.00 R-MMU-1169394 0.00000000  8.147701e-01      5
R-MMU-5654402.0.03 R-MMU-1169398 0.20638889  6.320604e-01      5
R-MMU-5654404.0.00 R-MMU-1169402 0.00000000  6.215393e-01      6
R-MMU-5654404.0.03 R-MMU-1169403 0.29472222  5.347271e-01      4
R-MMU-5654406.0.00 R-MMU-

R-MMU-5654653.0.03 R-MMU-2395801 0.14194444  9.861510e-02     12
R-MMU-5654655.0.00 R-MMU-2396594 0.00000000  9.251599e-02     19
R-MMU-5654655.0.03 R-MMU-2424480 0.17666667  3.738862e-01     15
R-MMU-5654658.0.00 R-MMU-2424482 0.00000000  3.738862e-01     15
R-MMU-5654658.0.03 R-MMU-2424484 0.17666667  1.847667e-01     22
R-MMU-5654659.0.00 R-MMU-2424486 0.00000000  1.847667e-01     22
R-MMU-5654659.0.03 R-MMU-2424487 0.14166667  1.847667e-01     22
R-MMU-5654662.0.00  R-MMU-388814 0.00000000  1.758400e-01      6
R-MMU-5654662.0.03  R-MMU-388829 0.10944444  1.714377e-01      9
R-MMU-5654663.0.00  R-MMU-388831 0.00000000  1.827546e-01      8
R-MMU-5654663.0.03  R-MMU-388832 0.11944444  3.876870e-01      9
R-MMU-5654664.0.00  R-MMU-388833 0.00000000  6.142094e-01     17
R-MMU-5654664.0.03  R-MMU-389158 0.14305556  3.876870e-01      9
R-MMU-5654667.0.00  R-MMU-389348 0.00000000  2.759003e-01      8
R-MMU-5654667.0.03  R-MMU-389350 0.13166667  2.650738e-01      8
R-MMU-5654669.0.00  R-MMU

R-MMU-9619682.0.03 R-MMU-9690410 0.30055556 -3.965819e-02      2
R-MMU-9010527.0.00 R-MMU-9690416 0.00000000  1.476424e-02      4
R-MMU-9010527.0.03 R-MMU-9690868 0.41750000  1.957409e-01      3
R-MMU-9011075.0.00 R-MMU-9690898 0.00000000  1.957409e-01      3
R-MMU-9011075.0.03 R-MMU-9692133 0.32583333  6.004733e-02      2
R-MMU-9011080.0.00 R-MMU-9692136 0.00000000  1.434296e-02      2
R-MMU-9011080.0.03 R-MMU-9692137 0.29194444  1.156662e-01      2
R-MMU-9011082.0.00 R-MMU-9692138 0.00000000  1.156662e-01      2
R-MMU-9011082.0.03 R-MMU-9692139 0.29194444  6.004733e-02      2
R-MMU-9011144.0.00 R-MMU-9692145 0.00000000  1.434296e-02      2
R-MMU-9011144.0.03 R-MMU-9692156 0.29944444  1.510815e-01      4
R-MMU-9011146.0.00 R-MMU-9692157 0.00000000  1.510815e-01      4
R-MMU-9011146.0.03 R-MMU-9692268 0.29944444 -3.965819e-02      2
R-MMU-9753028.0.00  R-MMU-450434 0.00000000  8.040181e-02      1
R-MMU-9753028.0.03  R-MMU-450466 0.25583333  9.048513e-01     45
R-MMU-9753126.0.00  R-MMU

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 0.27444444  7.035185e-01      8
R-MMU-73632.0.00   R-MMU-8876193 0.00000000  7.822476e-01      8
R-MMU-73632.0.03   R-MMU-8876454 0.32416667  4.576238e-01      6
R-NUL-8963657.0.00 R-MMU-8876616 0.00000000  3.950241e-01      7
R-NUL-8963657.0.03 R-MMU-8876837 0.32416667  4.163511e-01      8
R-NUL-9616935.0.00 R-MMU-8877308 0.00000000  6.663746e-01      7
R-NUL-9616935.0.03 R-MMU-8877311 0.32416667  6.080485e-01      7
R-MMU-9638179.0.00 R-MMU-8877451 0.00000000  4.283350e-01      9
R-MMU-9638179.0.03 R-MMU-8877760 0.29722222  3.422331e-01      8
R-MMU-158832.0.00  R-MMU-8877813 0.00000000  5.516996e-01      8
R-MMU-158832.0.03  R-MMU-8877998 0.36194444  5.737946e-01      7
R-MMU-174967.0.00  R-MMU-5685604 0.00000000 -4.891366e-02      1
R-MMU-174967.0.03  R-MMU-9647982 0.36194444  4.652354e-01      4
R-MMU-9753676.0.00 R-MMU-5694082 0.00000000  2.600067e-02      3
R-MMU-9753676.0.03 R-MMU-2022378 0.36194444  1.656129e-02      2
R-MMU-9708466.0.00 R-MMU-2022379 0.00000000  1.656129e-02

R-MMU-5432825.0.03 R-MMU-9034042 0.21861111  4.848902e-01      1
R-MMU-5432849.0.00 R-MMU-9034053 0.00000000  4.635924e-01      1
R-MMU-5432849.0.03   R-MMU-71130 0.22750000  2.860068e-01      1
R-MMU-6805943.0.00  R-MMU-391372 0.00000000  1.782700e-01      2
R-MMU-6805943.0.03 R-MMU-2681681 0.20833333  2.241764e-01      5
R-MMU-8870678.0.00  R-MMU-507719 0.00000000  3.985892e-01      4
R-MMU-8870678.0.03 R-MMU-5205649 0.16638889  7.191923e-01      7
R-MMU-936985.0.00  R-MMU-5205663 0.00000000  8.492958e-01      9
R-MMU-936985.0.03  R-MMU-5205673 0.22750000  7.133616e-01      5
R-MMU-9707594.0.00 R-MMU-9759158 0.00000000  1.846503e-02      1
R-MMU-9707594.0.03 R-MMU-8932375 0.28694444  5.944177e-01      4
R-MMU-9707659.0.00 R-MMU-9760081 0.00000000  3.802977e-02      2
R-MMU-9707659.0.03 R-MMU-9760082 0.25194444  3.802977e-02      2
R-MMU-422454.0.00  R-MMU-9762093 0.00000000  7.558565e-01      4
R-MMU-422454.0.03  R-MMU-9762094 0.28694444  9.615019e-02      2
R-MMU-879584.0.00  R-MMU-

R-MMU-203613.0.03  R-MMU-9822463 0.26194444  1.151834e-01     15
R-MMU-9021966.0.00 R-MMU-9822592 0.00000000  4.218352e-01      3
R-MMU-9021966.0.03 R-MMU-9822631 0.33277778  5.458453e-01     39
R-MMU-9021969.0.00 R-MMU-9822637 0.00000000  5.458453e-01     38
R-MMU-9021969.0.03 R-MMU-9822924 0.33277778  1.361342e-01     15
R-MMU-5683925.0.00 R-MMU-9822964 0.00000000  2.268061e-01     15
R-MMU-5683925.0.03  R-MMU-997263 0.09916667  1.879062e-01     15
R-MMU-880002.0.00  R-MMU-8943987 0.00000000  7.147252e-02      2
R-MMU-880002.0.03  R-MMU-1678841 0.25222222  5.882104e-01     10
R-MMU-880033.0.00   R-MMU-202212 0.00000000  1.373238e-01      2
R-MMU-880033.0.03   R-MMU-202354 0.25222222 -9.002634e-02      1
R-MMU-5362518.0.00  R-MMU-202407 0.00000000 -8.989349e-02      2
R-MMU-5362518.0.03  R-MMU-210283 0.19750000 -1.385100e-02      2
R-MMU-74872.0.00    R-MMU-212706 0.00000000  2.943245e-01      9
R-MMU-74872.0.03    R-MMU-212710 0.25222222  2.943245e-01      9
R-MMU-9830929.0.00  R-MMU

R-MMU-1855232.0.03  R-MMU-975919 0.39888889  2.206078e-01      6
R-MMU-70609.0.00   R-MMU-9851347 0.00000000  1.189092e-01      2
R-MMU-70609.0.03    R-MMU-186663 0.24111111  4.445874e-01      2
R-MMU-1676162.0.00 R-MMU-9832693 0.00000000  3.760918e-01      1
R-MMU-1676162.0.03 R-MMU-9832702 0.29638889  4.200279e-01      2
R-MMU-1676164.0.00  R-MMU-192157 0.00000000  9.632292e-02      2
R-MMU-1676164.0.03  R-MMU-193709 0.29638889  9.632292e-02      2
R-MMU-1855180.0.00  R-MMU-193845 0.00000000  9.632292e-02      2
R-MMU-1855180.0.03   R-MMU-76496 0.29638889  9.632292e-02      2
R-MMU-1855202.0.00 R-MMU-9018807 0.00000000  8.073389e-03      2
R-MMU-1855202.0.03 R-MMU-9714694 0.29638889  7.957955e-02      1
R-MMU-2172433.0.00  R-MMU-198824 0.00000000  9.592145e-01      2
R-MMU-2172433.0.03 R-MMU-8936442 0.15000000  8.749470e-01      4
R-MMU-8944233.0.00 R-MMU-3318415 0.00000000  3.957722e-01     32
R-MMU-8944233.0.03 R-MMU-3769447 0.17222222  6.297127e-01     84
R-MMU-8944229.0.00 R-MMU-

In [29]:
%%R
pid <- read.table(paste(OUT_DIR,"tcdd_project_id0.03vs0_time_cross.txt",sep=""),header=FALSE)
sid <- read.table(paste(OUT_DIR,"tcdd_sample_id0.03vs0_time_cross.txt",sep=""), header=FALSE)
sex <- read.table(paste(OUT_DIR, "tcdd_gender0.03vs0_time_cross.txt", sep=""), header=FALSE)
print(sex)

In [31]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = tcdd_dose_detail.vec.train,
   Project_id = pid$V1,
   Sample_id = sid$V1
)

ggplot(pca.d, aes(x = PC1, y = PC2, colour = Section)) +
   geom_point(size = 3) +  # 绘制点，指定点的大小
   geom_text(aes(label = paste(Project_id, Sample_id)), vjust = -1, hjust = 1, size = 2) +  # 添加标签
   scale_colour_manual(values = c("0" = "red", "0.03" = "green")) +  # 手动指定颜色
   theme_bw()

p <- plot_ly(
   data = pca.d,
   x = ~PC1,
   y = ~PC2,
   z = ~PC3,
   type = "scatter3d",
   mode = "markers",
   marker = list(
     color = as.factor(pca.d$Section),  # 将 Section 转换为因子
     colors = c("red", "green"),  # 定义颜色
     size = 5
   ),
   text = ~paste("Project ID:", Project_id, "Sample ID:", Sample_id)  # 在悬停文本中显示 Project ID
)
saveWidget(p, "plotly_chart0.03vs0_time_cross.html", selfcontained = TRUE)